# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298408622586                   -0.85    5.4         
  2   -8.300203661017       -2.75       -1.25    1.0    100ms
  3   -8.300433681552       -3.64       -1.89    1.5    106ms
  4   -8.300462271955       -4.54       -2.78    3.4    136ms
  5   -8.300464391961       -5.67       -3.12    2.6    135ms
  6   -8.300464519631       -6.89       -3.27    4.5    141ms
  7   -8.300464584565       -7.19       -3.41    2.9    172ms
  8   -8.300464616999       -7.49       -3.56    1.4    105ms
  9   -8.300464636868       -7.70       -3.75    5.6    209ms
 10   -8.300464640054       -8.50       -3.86    1.0    110ms
 11   -8.300464643424       -8.47       -4.12    1.5    162ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67458071993                   -0.70    6.1         
  2   -16.67874485238       -2.38       -1.14    1.5    269ms
  3   -16.67921962233       -3.32       -1.87    3.2    318ms
  4   -16.67928062738       -4.21       -2.77    3.2    306ms
  5   -16.67928600642       -5.27       -3.19    4.2    397ms
  6   -16.67928620427       -6.70       -3.49    2.6    312ms
  7   -16.67928621934       -7.82       -3.99    1.5    246ms
  8   -16.67928622153       -8.66       -4.60    2.8    286ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32640457204                   -0.56    7.4         
  2   -33.33424187779       -2.11       -1.00    1.4    987ms
  3   -33.33594776323       -2.77       -1.72    5.5    1.32s
  4   -33.33612677224       -3.75       -2.63    2.9    1.14s
  5   -33.33677465250       -3.19       -2.21    9.1    1.71s
  6   -33.33681075698       -4.44       -2.27    2.0    940ms
  7   -33.33694265186       -3.88       -3.33    1.5    1.03s
  8   -33.33694371716       -5.97       -3.70    4.4    1.46s
  9   -33.33694376340       -7.33       -3.93    2.2    978ms
 10   -33.33694378175       -7.74       -4.70    2.5    1.02s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298347375806                   -0.85    5.1         
  2   -8.300243359476       -2.72       -1.59    1.5   90.8ms
  3   -8.300443888388       -3.70       -2.71    2.9    111ms
  4   -8.300450406783       -5.19       -2.68    3.5    153ms
  5   -8.300464309274       -4.86       -3.34    1.0   82.1ms
  6   -8.300464585283       -6.56       -3.82    2.0    102ms
  7   -8.300464642014       -7.25       -4.56    1.6    117ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32565741299                   -0.56    7.1         
  2   -33.29453491972   +   -1.51       -1.26    1.0    769ms
  3   +3.716626419641   +    1.57       -0.28    6.5    2.15s
  4   -33.33423370751        1.57       -1.93    6.2    2.08s
  5   -33.29230528781   +   -1.38       -1.57    4.8    1.60s
  6   -32.97881495931   +   -0.50       -1.28    4.0    1.38s
  7   -33.21911007801       -0.62       -1.50    4.5    1.31s
  8   -33.33403595420       -0.94       -2.10    4.0    1.30s
  9   -33.33392008663   +   -3.94       -2.16    2.2    1.01s
 10   -33.33684830296       -2.53       -2.81    2.0    976ms
 11   -33.33691512747       -4.18       -3.10    3.1    1.21s
 12   -33.33693722511       -4.66      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.